In [1]:
#Second question in our analysis of San Fransisco building permits:
#2) What was the most likely kind of completed permit, per neighborhood, in San Francisco?

#This analysis should produce a dataframe of two columns, the first detailing each neighborhood, and the second
#detailing the most common completed permit for that neighborhood. Let's get started!

In [116]:
import pandas as pd

In [118]:
sf_permits = pd.read_csv("Building_Permits.csv")

/Users/Admin/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (22,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [179]:
pd.set_option('display.max_columns', 500)
sf_permits.head()

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
0,201505065519,4,sign - erect,05/06/2015,0326,023,140,NaN,Ellis,St,NaN,NaN,"ground fl facade: to erect illuminated, electr...",expired,12/21/2017,05/06/2015,11/09/2015,NaN,11/09/2015,NaN,6.0,NaN,NaN,NaN,11/03/2016,4000.0,4000.0,tourist hotel/motel,143.0,NaN,NaN,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.785719256680785, -122.40852313194863)",1380611233945
1,201604195146,4,sign - erect,04/19/2016,0306,007,440,NaN,Geary,St,0.0,NaN,remove (e) awning and associated signs.,issued,08/03/2017,04/19/2016,08/03/2017,NaN,08/03/2017,NaN,7.0,NaN,NaN,NaN,12/03/2017,1.0,500.0,tourist hotel/motel,NaN,NaN,NaN,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,3.0,Tenderloin,94102.0,"(37.78733980600732, -122.41063199757738)",1420164406718
2,201605278609,3,additions alterations or repairs,05/27/2016,0595,203,1647,NaN,Pacific,Av,NaN,NaN,installation of separating wall,withdrawn,09/26/2017,05/27/2016,NaN,NaN,NaN,NaN,6.0,6.0,NaN,NaN,NaN,20000.0,NaN,retail sales,39.0,retail sales,39.0,2.0,NaN,1.0,constr type 1,1.0,constr type 1,NaN,3.0,Russian Hill,94109.0,"(37.7946573324287, -122.42232562979227)",1424856504716
3,201611072166,8,otc alterations permit,11/07/2016,0156,011,1230,NaN,Pacific,Av,0.0,NaN,repair dryrot & stucco at front of bldg.,complete,07/24/2017,11/07/2016,07/18/2017,07/24/2017,07/18/2017,NaN,2.0,2.0,NaN,NaN,07/13/2018,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,2.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,3.0,Nob Hill,94109.0,"(37.79595867909168, -122.41557405519474)",1443574295566
4,201611283529,6,demolitions,11/28/2016,0342,001,950,NaN,Market,St,NaN,NaN,demolish retail/office/commercial 3-story buil...,issued,12/01/2017,11/28/2016,12/01/2017,NaN,11/20/2017,NaN,3.0,NaN,NaN,NaN,12/01/2018,100000.0,100000.0,retail sales,NaN,NaN,NaN,2.0,NaN,3.0,constr type 3,NaN,NaN,NaN,6.0,Tenderloin,94102.0,"(37.78315261897309, -122.40950883997789)",144548169992


In [7]:
#The columns we'll want to isolate are Permit Number, Permit Type Definition, Current Status, and 
#Neighborhoods - Analysis Boundaries. Let's go ahead and extract those into a new dataframe. 

In [122]:
dfa = sf_permits[["Permit Number", "Permit Type Definition", "Current Status", "Neighborhoods - Analysis Boundaries"]]
dfa.head()

,Permit Number,Permit Type Definition,Current Status,Neighborhoods - Analysis Boundaries
0,201505065519,sign - erect,expired,Tenderloin
1,201604195146,sign - erect,issued,Tenderloin
2,201605278609,additions alterations or repairs,withdrawn,Russian Hill
3,201611072166,otc alterations permit,complete,Nob Hill
4,201611283529,demolitions,issued,Tenderloin


In [180]:
#Great. Now let's remove the spaces in the column headers and make some shorter so they're easier to work with. 
dfb = dfa.rename(columns={"Permit Number" : "Permit_Number", "Permit Type Definition" : "Permit_Type_Definition", "Current Status" : "Current_Status", "Neighborhoods - Analysis Boundaries" : "Neighborhood"})
dfb.head()

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
0,201505065519,sign - erect,expired,Tenderloin
1,201604195146,sign - erect,issued,Tenderloin
2,201605278609,additions alterations or repairs,withdrawn,Russian Hill
3,201611072166,otc alterations permit,complete,Nob Hill
4,201611283529,demolitions,issued,Tenderloin


In [124]:
#Now let's make a new dataframe where we only see records that have a "complete" current status.
dfc = dfb.loc[dfb['Current_Status'] == 'complete']
dfc.head()

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
3,201611072166,otc alterations permit,complete,Nob Hill
6,201706300814,otc alterations permit,complete,Inner Sunset
48,201710110928,otc alterations permit,complete,Portola
91,201407080724,additions alterations or repairs,complete,Portola
92,201408254591,otc alterations permit,complete,South of Market


In [125]:
#Great! Now let's check for any missing values.
dfc.isnull().values.any()

True

In [126]:
#Okay, there seems to be missing values. Where are they and how many are there in each column?
null_columns = dfc.columns[dfc.isnull().any()]
dfc[null_columns].isnull().sum()

Neighborhood    746
dtype: int64

In [127]:
#So looks like there's 746, only in the Neighborhood column. We don't want nulls in that column, 
#so let's get rid of all rows where there's a null in the Neighborhood column.
dfd = dfc.dropna()
dfd.isnull().values.any()

False

In [128]:
#Great. Now let's check for duplicates by looking at the information contained in the describe() method.
dfd.describe()

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
count,96331,96331,96331,96331
unique,88167,8,1,41
top,201602179758,otc alterations permit,complete,Financial District/South Beach
freq,66,89734,96331,13095


In [184]:
#It seems that we have some duplicates in Permit Number.
#Let's verify that Permit_Type_Definition and Neighborhod should have 8 and 41 unique values respectively.
len(dfd['Permit_Type_Definition'].unique())

8

In [185]:
#Great. And:
len(dfd['Neighborhood'].unique())

41

In [181]:
#Awesome. Now let's take a look at the Permit_Number column more carefully. How many duplicated permit numbers are
#there?

96331-88167 

8164

In [130]:
#And how many rows in the entire dataframe are exactly the same in EACH column? 
dfd.duplicated().sum()

8161

In [ ]:
#This means that 8161 rows in the dataframe are the exact same. We should get rid of those first. 
#After that, we'll still have 3 remaining which probably have different permit types or neighborhood. We'll have to
#look at those later. 

In [131]:
#First, Let's drop the 8,161 rows that are exactly the same.
dfe = dfd.drop_duplicates()
dfe.describe()

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
count,88170,88170,88170,88170
unique,88167,8,1,41
top,201312113897,otc alterations permit,complete,Financial District/South Beach
freq,2,82383,88170,12915


In [132]:
#Great. So now there's only 3 duplicates that we need to look into. Let's find them.
dfe.loc[dfe.Permit_Number.duplicated(), :]


,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
33531,201312113897,otc alterations permit,complete,Sunset/Parkside
35997,201401095948,additions alterations or repairs,complete,Lakeshore
62898,201410017819,otc alterations permit,complete,Sunset/Parkside


In [133]:
#So it looks like these 3 permit numbers: 201312113897, 201401095948, and 201410017819 have duplicates somewhere in
#the dataframe, dfe. Let's find them one at a time, and see how they compare, starting with the first.

dfe.loc[dfe.Permit_Number == "201312113897"]

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
33530,201312113897,otc alterations permit,complete,Mission
33531,201312113897,otc alterations permit,complete,Sunset/Parkside


In [134]:
#So they're completely the same, except for the neighborhood. Let's look to see if they were completed on different 
#dates. If that's the case, maybe we can keep the entry for that permit # with the most recent completed date.
sf_permits.loc[sf_permits['Permit Number'] == "201312113897"]

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
33530,201312113897,8,otc alterations permit,12/11/2013,4085,034,3031,NaN,20th,St,NaN,NaN,to comply with physical inspection report # 3r...,complete,01/14/2014,12/11/2013,12/11/2013,01/14/2014,12/11/2013,NaN,2.0,2.0,NaN,NaN,12/06/2014,1.0,1.0,2 family dwelling,2.0,2 family dwelling,2.0,0.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,9.0,Mission,94110.0,"(37.75886763017198, -122.4113202595743)",1326742169954
33531,201312113897,8,otc alterations permit,12/11/2013,7218,003,3035,NaN,20th,Av,NaN,NaN,to comply with physical inspection report # 3r...,complete,01/14/2014,12/11/2013,12/11/2013,01/14/2014,12/11/2013,NaN,2.0,2.0,NaN,NaN,12/06/2014,1.0,1.0,2 family dwelling,2.0,2 family dwelling,2.0,0.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,7.0,Sunset/Parkside,94132.0,"(37.732029263768375, -122.47628839261105)",1326743224419


In [102]:
#Ah, so they both have the same Completed Date. It looks like whoever entered the data made a mistake here, 
#since we have the same 2 Permit Numbers with the same Completed Date, but differing Neighborhoods. 
#I think it's best to just get rid of all entries with that Permit Number for the sake of the analysis. 
#Let's check this for the other two permit numbers. 

dfe.loc[dfe.Permit_Number == "201401095948"]

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
35996,201401095948,additions alterations or repairs,complete,Oceanview/Merced/Ingleside
35997,201401095948,additions alterations or repairs,complete,Lakeshore


In [103]:
#Same Completed Date here too. Let's get rid of this. On to the next one!
sf_permits.loc[sf_permits['Permit Number'] == "201401095948"]

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
35996,201401095948,3,additions alterations or repairs,01/09/2014,7060,021,50,NaN,Summit,St,NaN,NaN,install overhead firesprinkler system per nfpa...,complete,12/19/2014,01/09/2014,03/21/2014,12/19/2014,03/21/2014,NaN,NaN,4.0,NaN,Y,03/16/2015,25000.0,25000.0,NaN,NaN,2 family dwelling,2.0,2.0,NaN,NaN,NaN,5.0,wood frame (5),NaN,11.0,Oceanview/Merced/Ingleside,94112.0,"(37.7159082469054, -122.45345239839023)",1329352219700
35997,201401095948,3,additions alterations or repairs,01/09/2014,7331,005,52,NaN,Summit,Wy,NaN,NaN,install overhead firesprinkler system per nfpa...,complete,12/19/2014,01/09/2014,03/21/2014,12/19/2014,03/21/2014,NaN,NaN,4.0,NaN,Y,03/16/2015,25000.0,25000.0,NaN,NaN,2 family dwelling,2.0,2.0,NaN,NaN,NaN,5.0,wood frame (5),NaN,7.0,Lakeshore,94132.0,"(37.714814512478924, -122.47582502606711)",1329353495041


In [112]:
dfe.loc[dfe.Permit_Number == "201410017819"]

,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
62897,201410017819,otc alterations permit,complete,Outer Richmond
62898,201410017819,otc alterations permit,complete,Sunset/Parkside


In [136]:
#Same thing here too! Looks like we'll have to get rid of all three permit numbers. 
sf_permits.loc[sf_permits['Permit Number'] == "201410017819"]

,Permit Number,Permit Type,Permit Type Definition,Permit Creation Date,Block,Lot,Street Number,Street Number Suffix,Street Name,Street Suffix,Unit,Unit Suffix,Description,Current Status,Current Status Date,Filed Date,Issued Date,Completed Date,First Construction Document Date,Structural Notification,Number of Existing Stories,Number of Proposed Stories,Voluntary Soft-Story Retrofit,Fire Only Permit,Permit Expiration Date,Estimated Cost,Revised Cost,Existing Use,Existing Units,Proposed Use,Proposed Units,Plansets,TIDF Compliance,Existing Construction Type,Existing Construction Type Description,Proposed Construction Type,Proposed Construction Type Description,Site Permit,Supervisor District,Neighborhoods - Analysis Boundaries,Zipcode,Location,Record ID
62897,201410017819,8,otc alterations permit,10/01/2014,1405,028,390,NaN,29th,Av,NaN,NaN,replace 2 windows in front of house.,complete,10/20/2014,10/01/2014,10/01/2014,10/20/2014,10/01/2014,NaN,2.0,2.0,NaN,NaN,09/26/2015,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,0.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,1.0,Outer Richmond,94121.0,"(37.78201526219179, -122.48890430682486)",1357457242083
62898,201410017819,8,otc alterations permit,10/01/2014,2400,006,2439,NaN,25th,Av,NaN,NaN,replace 2 windows in front of house.,complete,10/20/2014,10/01/2014,10/01/2014,10/20/2014,10/01/2014,NaN,2.0,2.0,NaN,NaN,09/26/2015,2000.0,2000.0,1 family dwelling,1.0,1 family dwelling,1.0,0.0,NaN,5.0,wood frame (5),5.0,wood frame (5),NaN,4.0,Sunset/Parkside,94116.0,"(37.741988921671314, -122.48239591666385)",1357458134445


In [137]:
#So now let's drop all entries with these 3 permit numbers in our dataframe, dfe. 
#dfe = dfe.drop(dfe.index[[33530, 33531, 35996, 35997, 62897, 62898]])
dfe = dfe[dfe.Permit_Number != '201312113897']
dfe = dfe[dfe.Permit_Number != '201401095948']
dfe = dfe[dfe.Permit_Number != '201410017819']
dfe.describe()


,Permit_Number,Permit_Type_Definition,Current_Status,Neighborhood
count,88164,88164,88164,88164
unique,88164,8,1,41
top,201401236927,otc alterations permit,complete,Financial District/South Beach
freq,1,82379,88164,12915


In [140]:
#Great. Now our data is clean and ready for us to work with. Let's get rid of the Current_Status column first; we don't
#need that anymore. 

dfe = dfe.drop(["Current_Status"], axis=1)
dfe.head()

,Permit_Number,Permit_Type_Definition,Neighborhood
3,201611072166,otc alterations permit,Nob Hill
6,201706300814,otc alterations permit,Inner Sunset
48,201710110928,otc alterations permit,Portola
91,201407080724,additions alterations or repairs,Portola
92,201408254591,otc alterations permit,South of Market


In [173]:
#Now, let's group this dataframe by Permit_Type and Neighborhood and count up the permit numbers in each category.
dff = dfe.groupby(['Neighborhood','Permit_Type_Definition']).agg('count').reset_index()
dfg = dff.rename(columns={"Permit_Type_Definition" : "Permit_Type", "Permit_Number" : "Number_of_Permits"})
dfg.head()


,Neighborhood,Permit_Type,Number_of_Permits
0,Bayview Hunters Point,additions alterations or repairs,144
1,Bayview Hunters Point,demolitions,32
2,Bayview Hunters Point,grade or quarry or fill or excavate,8
3,Bayview Hunters Point,new construction,5
4,Bayview Hunters Point,new construction wood frame,14


In [178]:
#Now, for this question, we only want to know the most common type of completed permit for each neighborhood, so let's
#only consider the max, or the highest value of permits per permit type per neighborhood. And Voila! Theres
dfh = dfg.groupby(["Neighborhood"]).max()["Permit_Type"].reset_index()
dfh.rename(columns={"Permit_Type" : "Most Common Type of Permit"})

,Neighborhood,Most Common Type of Permit
0,Bayview Hunters Point,wall or painted sign
1,Bernal Heights,sign - erect
2,Castro/Upper Market,wall or painted sign
3,Chinatown,wall or painted sign
4,Excelsior,wall or painted sign
5,Financial District/South Beach,wall or painted sign
6,Glen Park,otc alterations permit
7,Golden Gate Park,otc alterations permit
8,Haight Ashbury,wall or painted sign
9,Hayes Valley,wall or painted sign


In [ ]:
#Awesome. We're done! Just looking at the table, it looks like "wall or painted sign" was the most common permit type
#for many of the neighborhoods. Now let's export this dataframe into a csv. 
dfh.to_csv("Most_Common_Comp.Permits_by_Neighborhood.csv")